## Libraries, API Key

Don't forget to insert your Google API key

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import os

GOOGLE_API_KEY = ""
gmaps = googlemaps.Client(key = GOOGLE_API_KEY) 

np.random.seed(123)

## Constants

Import folder - the one with files like Leuven_aeds.csv (we have to make names more clear in final version)
Export folder - any folder you want

RATIO_EMERGENCY_NORMAL = 0.65 - this means that ambulance is 35% faster than a regular car for the same distance

In [2]:
import_folder_path = "/Users/Jovan/Desktop/MDA_Project/DataSegmented"
export_folder_path = "/Users/Jovan/Desktop/MDA_Project/DistanceMatrices"

cities = ["Antwerpen", "Brugge", "Brussels", "Charleroi", "Gent", "Leuven", "Liege", "Oostende"]

RATIO_EMERGENCY_NORMAL = 0.65

## Functions

In [3]:
# Adjusting coordinates so they suit directions() function

def format_coordinates(latitude, longitude):
    return f"{latitude}, {longitude}"

In [4]:
# Calculates distance matrix for given sets of origin and destination coordinates

def get_distance_matrix(origins, destinations, mode, coef = 1):
    num_origins = len(origins)
    num_destinations = len(destinations)
    result_matrix = np.zeros((num_origins, num_destinations))

    for i in range(0, num_origins):
        for j in range(0, num_destinations):
            orig = format_coordinates(origins.iloc[i, 0], origins.iloc[i, 1])
            dest = format_coordinates(destinations.iloc[j, 0], destinations.iloc[j, 1])
            print("Calculating pair: [", i, ",", j, "]")
            result_matrix[i][j] = gmaps.directions(orig, dest, mode)[0]['legs'][0]['duration']['value']
            print("Time:", result_matrix[i][j], "\n")

    return(result_matrix * coef)

### Import data

In [ ]:
os.chdir(import_folder_path)
city = "Leuven"

aeds = pd.read_csv(city + "_aeds.csv")
cards = pd.read_csv(city + "_cards.csv")
vehicles = pd.read_csv(city + "_vehicles.csv")

### Mandatory column

In [ ]:
aeds['public'] = aeds['public'].fillna(0)
aeds['public'] = ~aeds['public'].astype(bool)
aeds = aeds.rename(columns={'public': 'mandatory'})

vehicles['mandatory'] = 1

### Distance Matrix

In [ ]:
aeds_coordinates = aeds[['latitude', 'longitude']]
vehicles_coordinates = vehicles[['latitude', 'longitude']]
cards_coordinates = cards[['latitude', 'longitude']]

In [ ]:
aeds_dm = get_distance_matrix(aed_coordinates, cards_coordinates, "walking")
vehicles_dm = get_distance_matrix(vehicles_coordinates, cards_coordinates, "driving", RATIO_EMERGENCY_NORMAL)

In [ ]:
aeds_dm = np.hstack((aeds_dm, aeds['mandatory'].values.reshape(-1, 1)))
vehicles_dm = np.hstack((vehicles_dm, vehicles['mandatory'].values.reshape(-1, 1)))

In [ ]:
distance_matrix = np.vstack((aeds_dm, vehicles_dm))

### Export

In [ ]:
os.chdir(export_folder_path)

df = pd.DataFrame(distance_matrix)
df.to_csv("Leuven_distance_matrix.csv", index=False)

## Loop

In [5]:
for city in cities:
    # Import data on the city
    os.chdir(import_folder_path)
    aeds = pd.read_csv(city + "_aeds.csv")
    cards = pd.read_csv(city + "_cards.csv")
    vehicles = pd.read_csv(city + "_vehicles.csv")
    print("Importing " + city + " successful.")

    # Make moveable columns for aeds and vehicles
    # For AEDs: moveable is 1 if public, moveable is 0 if private or unknown
    # For vehicles: moveable is always 0
    aeds['public'] = aeds['public'].fillna(0)
    aeds['public'] = ~aeds['public'].astype(bool)
    aeds = aeds.rename(columns={'public': 'mandatory'})
    vehicles['mandatory'] = 1
    print("Made both 'mandatory' columns.\n")

    # Prepare distance matrix
    aeds_coordinates = aeds[['latitude', 'longitude']]
    vehicles_coordinates = vehicles[['latitude', 'longitude']]
    cards_coordinates = cards[['latitude', 'longitude']]

    print("Calculating aeds-cards walking times...\n")
    aeds_dm = get_distance_matrix(aeds_coordinates, cards_coordinates, "walking")
    print("Calculating vehicles-cards driving times...\n")
    vehicles_dm = get_distance_matrix(vehicles_coordinates, cards_coordinates, "driving", RATIO_EMERGENCY_NORMAL)

    aeds_dm = np.hstack((aeds_dm, aeds['mandatory'].values.reshape(-1, 1)))
    vehicles_dm = np.hstack((vehicles_dm, vehicles['mandatory"'].values.reshape(-1, 1)))
    distance_matrix = np.vstack((aeds_dm, vehicles_dm))

    # Export
    os.chdir(export_folder_path)
    df = pd.DataFrame(distance_matrix)
    df.to_csv(city + "_distance_matrix.csv", index=False)

Importing Antwerpen successful.
Made both 'mandatory' columns.

Calculating aeds-cards walking times...

Calculating pair: [ 0 , 0 ]
Time: 4244.0 

Calculating pair: [ 0 , 1 ]
Time: 4244.0 

Calculating pair: [ 0 , 2 ]
Time: 5782.0 

Calculating pair: [ 0 , 3 ]
Time: 5782.0 

Calculating pair: [ 0 , 4 ]
Time: 8587.0 

Calculating pair: [ 0 , 5 ]
Time: 8587.0 

Calculating pair: [ 0 , 6 ]
Time: 1668.0 

Calculating pair: [ 0 , 7 ]
Time: 3354.0 

Calculating pair: [ 0 , 8 ]
Time: 3354.0 

Calculating pair: [ 0 , 9 ]
Time: 3957.0 

Calculating pair: [ 0 , 10 ]
Time: 3957.0 

Calculating pair: [ 0 , 11 ]
Time: 3957.0 

Calculating pair: [ 0 , 12 ]
Time: 3361.0 

Calculating pair: [ 0 , 13 ]
Time: 3361.0 

Calculating pair: [ 0 , 14 ]
Time: 228.0 

Calculating pair: [ 0 , 15 ]
Time: 4982.0 

Calculating pair: [ 0 , 16 ]
Time: 4982.0 

Calculating pair: [ 0 , 17 ]
Time: 3724.0 

Calculating pair: [ 0 , 18 ]
Time: 3724.0 

Calculating pair: [ 0 , 19 ]
Time: 2772.0 

Calculating pair: [ 0 , 20

KeyboardInterrupt: 